## Introduction

### Understanding Bivariate Normal Distribution

Let $U$ and $V$ be two independent normal random variables, and consider two new random variables $X$ and $Y$ of the form
$$
\begin{aligned}
&X=a U+b V \\
&Y=c U+d V
\end{aligned}
$$
where $a, b, c, d$, are some scalars. Each one of the random variables $X$ and $Y$ is normal, since it is a linear function of independent normal random variables.Furthermore, because $X$ and $Y$ are linear functions of the same two independent normal random variables, their joint PDF takes a special form, known as the **bivariate normal PDF**. 

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
import time
import jax
from jax.random import multivariate_normal
import numpy as np
import plotly.graph_objects as go
from numpy import linalg as LA

In [2]:
# Add Sliders to change parameters
var_X = widgets.FloatSlider(description='var_X',min=1,value=1.5,step=0.1,continuous_update=False)
var_Y = widgets.FloatSlider(description='var_Y',min=1,value=1.0,step=0.1,continuous_update=False)
mean_X = widgets.FloatSlider(description='mean_X',value=0,step=0.1,continuous_update=False)
mean_Y = widgets.FloatSlider(description='mean_Y',value=0,step=0.1,continuous_update=False)
cov_XY = widgets.FloatSlider(description='cov_XY',value=0.4,step=0.1,continuous_update=False)
sample_size = widgets.IntSlider(description='sample_size',min=100,max=1000,continuous_update=False)

# Create a dict for easy access
slider_dict = {"var_X":var_X,
               "var_Y":var_Y,
               "mean_X":mean_X,
               "mean_Y":mean_Y,
               "cov_XY":cov_XY,
               "sample_size":sample_size}

## Generating Bivariate Normal Data usin JAX

$X$ and $Y$ are bivariately normally distributed with mean vector components $\mu_{1}$ and $\mu_{2}$ and variance-covariance matrix shown below:
$$
\left(\begin{array}{l}
X \\
Y
\end{array}\right) \sim N\left[\left(\begin{array}{l}
\mu_{1} \\
\mu_{2}
\end{array}\right),\left(\begin{array}{cc}
\sigma_{1}^{2} & \rho \sigma_{1} \sigma_{2} \\
\rho \sigma_{1} \sigma_{2} & \sigma_{2}^{2}
\end{array}\right)\right]
$$
In this case we have the variances for the two variables on the diagonal and on the off-diagonal we have the covariance between the two variables. This covariance is equal to the correlation($\rho$) times the product of the two standard deviations($\sigma_{1} and \ \sigma_{2}$) 

In [3]:
def generate_bivariate_norm():
    '''
    Add Function doc string
    '''
    
    key = jax.random.PRNGKey(0)
    cov = np.array([[var_X.value, cov_XY.value], [cov_XY.value, var_Y.value]])
    mean = np.array([mean_X.value,mean_Y.value])
    
    # Ensuirng the variance-covariance matrix is positive semidefinite usin eigen values, if not generate a new random positive 
    # semidefinite variance-covariance matrix
    
    cov_eignvals = LA.eigvals(cov)
    
    if(np.any(cov_eignvals < 0)):
        print(f"The current covariance matrix {cov} is not positive semidefinite hence a new random covariance martrix will be created")
        rand_mat = np.random.rand(2,2)
        cov = np.dot(rand_mat, rand_mat.T)
        cov_XY.value = cov[:, -1][0]
        
    x1,x2 = multivariate_normal(key, mean, cov, (sample_size.value,)).T
    return x1,x2

## Marginal PDF of random variables in Bivariate Normal Distribution

The marginal distributions of $N\left(\mu_{1}, \mu_{2}, \sigma_{1}^{2}, \sigma_{2}^{2}, \rho\right)$ are normal with r.v's $X$ and $Y$ having density functions
$$
f_{X}(x)=\frac{1}{\sqrt{2 \pi} \sigma_{1}} e^{-\frac{\left(x-\mu_{1}\right)^{2}}{2 \sigma_{1}^{2}}}, \quad f_{Y}(y)=\frac{1}{\sqrt{2 \pi} \sigma_{2}} e^{-\frac{\left(y-\mu_{2}\right)^{2}}{2 \sigma_{2}^{2}}}
$$


In [4]:
def marginal_PDF(x):
    mean = np.mean(x)
    var  = np.var(x)
    std_dev = np.sqrt(var)
    
    const = 1/(2.506628274631*std_dev)
    
    power = 1/np.exp(((x-mean)**2)/(2*var))
    return const*power

## Graphing and Plotting Data

### Intial Graph

In [5]:
# Generate a Bivariate Normal Distribution using JAX
x1,x2 = generate_bivariate_norm()

# Plot for marginal PDF of rv x1
fig_mar_x1 = go.Scatter3d(
    x=np.sort(x1),
    y=sample_size.value*[np.min(x2)],
    z=marginal_PDF(np.sort(x1)),
    mode='lines',
    marker=dict(
        size=2,
        # color=z,               
        colorscale='Viridis',   
        opacity=0.8
    ))

# Plot for marginal PDF of rv x2
fig_mar_x2 = go.Scatter3d(
    x=sample_size.value*[np.min(x1)],
    y=np.sort(x2),
    z=marginal_PDF(np.sort(x2)),
    mode='lines',
    marker=dict(
        size=2,
        # color=z,                
        colorscale='Viridis',   
        opacity=0.8
    ))

# Scatter Plot for both rv's
fig_XY = go.Scatter3d(
    x=x1,
    y=x2,
    z=sample_size.value*[0],
    mode='markers',
    marker=dict(
        size=2,
        # color=z,               
        colorscale='Viridis',   
        opacity=0.8
    ))


fig = go.FigureWidget(data=[fig_XY,fig_mar_x1,fig_mar_x2])

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0),
                    scene = dict(
                      #  xaxis = dict(nticks=4, range=[-100,100],),
                      #  yaxis = dict(nticks=4, range=[-50,100],),
                       zaxis = dict(range=[0,1]))
                    )


container_slider = widgets.VBox(list(slider_dict.values()))

### Function to handle parameter updates

- Add doc string
- Change graph legend
- Restrict the space the graph takes

In [6]:
def response(v):
    '''
    v : Add function doc string
    
    
    
    
    
    '''
    
    ## DEBUG : Remove before sending
    #print(var_X.value,var_Y.value,mean_X.value,mean_Y.value,cov_XY.value,sample_size.value)

    # Generate a Bivariate Normal Distribution using JAX
    x1,x2 = generate_bivariate_norm()
    

    with fig.batch_update():
        fig.data[0].x = x1
        fig.data[0].y = x2
        fig.data[0].z = sample_size.value*[0]
        
        fig.data[1].x = np.sort(x1)
        fig.data[1].y = sample_size.value*[np.min(x2)]
        fig.data[1].z = marginal_PDF(np.sort(x1))
        
        fig.data[2].x = sample_size.value*[np.min(x1)]
        fig.data[2].y = np.sort(x2)
        fig.data[2].z = marginal_PDF(np.sort(x2))
        


for slider in slider_dict.values():
    slider.observe(response, names='value')

widgets.VBox([container_slider,fig])

The current covariance matrix [[ 1.5 28.6]
 [28.6  1. ]] is not positive semidefinite hence a new random covariance martrix will be created
The current covariance matrix [[1.5 7.4]
 [7.4 1. ]] is not positive semidefinite hence a new random covariance martrix will be created
The current covariance matrix [[ 1.5 42.6]
 [42.6  1. ]] is not positive semidefinite hence a new random covariance martrix will be created


### Refrences


- <a href="https://en.wikipedia.org/wiki/Definite_matrix#Negative-definite.2C_semidefinite_and_indefinite_matrices">Positive Semideifinite Matrix<a/>

- <a href="https://stackoverflow.com/questions/619335/a-simple-algorithm-for-generating-positive-semidefinite-matrices">Creating a positive semidefinite matrix</a>
    
- <a href="https://www.cuemath.com/algebra/covariance-matrix/">Variance Covariance Matrix Info</a>

- Bivariate Normal Distribution
    - http://athenasc.com/Bivariate-Normal.pdf
    - https://webspace.maths.qmul.ac.uk/i.goldsheid/MTH5118/Notes11-09.pdf
    
**NOTE** : Refrences also include official documentation of JAX,plotly and ipywidgets